Karol Janic

kwiecień 2023

zad1.ipynb - Minimalizacja kosztu zakupu i dostawy paliwa dla lotnisk.

In [16]:
using JuMP
using GLPK

using JSON

In [17]:
dane_wejsciowe = JSON.parse(read("data1.json", String))

lotniska = dane_wejsciowe["lotniska"]
firmy = dane_wejsciowe["firmy"]
koszty = dane_wejsciowe["koszty"]

Dict{String, Any} with 4 entries:
  "Lotnisko 3" => Dict{String, Any}("Firma 1"=>9, "Firma 3"=>4, "Firma 2"=>12)
  "Lotnisko 2" => Dict{String, Any}("Firma 1"=>10, "Firma 3"=>14, "Firma 2"=>11)
  "Lotnisko 4" => Dict{String, Any}("Firma 1"=>11, "Firma 3"=>9, "Firma 2"=>13)
  "Lotnisko 1" => Dict{String, Any}("Firma 1"=>10, "Firma 3"=>8, "Firma 2"=>7)

In [18]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

Definicja zmiennych decyzyjnych(liczba galonów paliwa zamawianego przez dane lotnisko od danej firmy) oraz
ograniczenie ich wartości do liczb nieujemnych:

In [19]:
@variable(model, x[keys(firmy), keys(lotniska)] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["Firma 1", "Firma 3", "Firma 2"]
    Dimension 2, ["Lotnisko 3", "Lotnisko 2", "Lotnisko 4", "Lotnisko 1"]
And data, a 3×4 Matrix{VariableRef}:
 x[Firma 1,Lotnisko 3]  x[Firma 1,Lotnisko 2]  …  x[Firma 1,Lotnisko 1]
 x[Firma 3,Lotnisko 3]  x[Firma 3,Lotnisko 2]     x[Firma 3,Lotnisko 1]
 x[Firma 2,Lotnisko 3]  x[Firma 2,Lotnisko 2]     x[Firma 2,Lotnisko 1]

Ograniczenie zasobów firm:

In [20]:
@constraint(model, 
    [firma in keys(firmy)], sum(x[firma, :]) <= firmy[firma]["zasoby"])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["Firma 1", "Firma 3", "Firma 2"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 x[Firma 1,Lotnisko 3] + x[Firma 1,Lotnisko 2] + x[Firma 1,Lotnisko 4] + x[Firma 1,Lotnisko 1] ≤ 275000.0
 x[Firma 3,Lotnisko 3] + x[Firma 3,Lotnisko 2] + x[Firma 3,Lotnisko 4] + x[Firma 3,Lotnisko 1] ≤ 660000.0
 x[Firma 2,Lotnisko 3] + x[Firma 2,Lotnisko 2] + x[Firma 2,Lotnisko 4] + x[Firma 2,Lotnisko 1] ≤ 550000.0

Ograniczenie zapotrzebowań lotnisk:

In [21]:
@constraint(model, 
    [lotnisko in keys(lotniska)], sum(x[:, lotnisko]) >= lotniska[lotnisko]["zapotrzebowanie"])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["Lotnisko 3", "Lotnisko 2", "Lotnisko 4", "Lotnisko 1"]
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 x[Firma 1,Lotnisko 3] + x[Firma 3,Lotnisko 3] + x[Firma 2,Lotnisko 3] ≥ 330000.0
 x[Firma 1,Lotnisko 2] + x[Firma 3,Lotnisko 2] + x[Firma 2,Lotnisko 2] ≥ 220000.0
 x[Firma 1,Lotnisko 4] + x[Firma 3,Lotnisko 4] + x[Firma 2,Lotnisko 4] ≥ 440000.0
 x[Firma 1,Lotnisko 1] + x[Firma 3,Lotnisko 1] + x[Firma 2,Lotnisko 1] ≥ 110000.0

Funkcja kosztu w dolarach(liczba galonów zamawianego paliwa * cena kupna i dostarczenia jednego galonu paliwa w dolarach na galon)

In [22]:
@objective(model, Min, 
    sum(koszty[lotnisko][firma] * x[firma, lotnisko] for firma in keys(firmy), lotnisko in keys(lotniska)));

In [23]:
optimize!(model)

In [24]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.52500e+06
  Objective bound    : -Inf
  Dual objective value : 8.52500e+06

* Work counters
  Solve time (sec)   : 5.88894e-05


In [25]:
JuMP.objective_value(model)

8.525e6

In [26]:
for lotnisko in sort!(collect(keys(lotniska))), firma in sort!(collect(keys(firmy)))
    println(firma, " dostarcza ", value(x[firma, lotnisko]), " galonow paliwa dla ", lotnisko)
end

Firma 1 dostarcza 0.0 galonow paliwa dla Lotnisko 1
Firma 2 dostarcza 110000.0 galonow paliwa dla Lotnisko 1
Firma 3 dostarcza 0.0 galonow paliwa dla Lotnisko 1
Firma 1 dostarcza 165000.0 galonow paliwa dla Lotnisko 2
Firma 2 dostarcza 55000.0 galonow paliwa dla Lotnisko 2
Firma 3 dostarcza 0.0 galonow paliwa dla Lotnisko 2
Firma 1 dostarcza 0.0 galonow paliwa dla Lotnisko 3
Firma 2 dostarcza 0.0 galonow paliwa dla Lotnisko 3
Firma 3 dostarcza 330000.0 galonow paliwa dla Lotnisko 3
Firma 1 dostarcza 110000.0 galonow paliwa dla Lotnisko 4
Firma 2 dostarcza 0.0 galonow paliwa dla Lotnisko 4
Firma 3 dostarcza 330000.0 galonow paliwa dla Lotnisko 4


In [27]:
for firma in sort!(collect(keys(firmy)))
    println(firma, " dostarcza ", value(sum(x[firma, :])), " paliwa")
end

Firma 1 dostarcza 275000.0 paliwa
Firma 2 dostarcza 165000.0 paliwa
Firma 3 dostarcza 660000.0 paliwa


In [28]:
for firma in sort!(collect(keys(firmy)))
    if value(sum(x[firma, :])) < firmy[firma]["zasoby"]
        println(firma, " ma jescze paliwo")
    else 
        println(firma, " nie ma juz paliwa")
    end
end 

Firma 1 nie ma juz paliwa
Firma 2 ma jescze paliwo
Firma 3 nie ma juz paliwa
